In [30]:
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import mlx.core as mx
import mlx.optimizers as optim
from mlx.utils import tree_flatten
from mlx_lm import load, generate
from mlx_lm.tuner import train, evaluate, TrainingArgs
from mlx_lm.tuner import linear_to_lora_layers

import json
from pathlib import Path
import tqdm

In [2]:
dim_set = ['objectives', 'AI_app', 'Disability_type', 'Barriers', 'Facilitators', 'Equity',
           'Tools', 'Disability_outcome', 'Implications']

In [31]:
# Define model and adapter paths
model_path = "mistralai/Mistral-7B-Instruct-v0.2"
# Define LoRA configuration
lora_config = {
    "lora_layers": 8,
    "num_layers": 12,  # Replace with the actual number of layers for your model
    "lora_parameters": {
        "rank": 8,
        "scale": 10.0,
        "dropout": 0.1,
    },
}
# model_path = "mistralai/Mistral-7B-Instruct-v0.2" #"meta-llama/Llama-3.2-1B-Instruct"
# "microsoft/Phi-3-mini-4k-instruct"
model, tokenizer = load(model_path)


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [17]:
instruction1 = """[INST]You are an expert in data extraction of systematic reviews, Extract information on the specific applications or tools developed using AI that aim to improve the lives of persons with disabilities in the following full-text  This could include, but is not limited to: + Assistive technologies for daily living tasks (e.g., automated door openers, smart home devices) + Communication aids (e.g., speech-to-text systems, sign language recognition) + Diagnostic tools (e.g., image analysis software for visual impairments, audio processing software for hearing impairments) + Rehabilitation and therapy tools (e.g., virtual reality-based physical therapy, AI-powered cognitive training) just give it in one short phrase no additional explanations:"""
matn = """Automatic health record review to help prioritize gravely ill Social Security disability applicants
Abstract
Objective: Every year, thousands of patients die waiting for disability benefits from the Social Security Administration. Some qualify for expedited service under the Compassionate Allowance (CAL) initiative, but CAL software focuses exclusively on information from a single form field. This paper describes the development of a supplemental process for identifying some overlooked but gravely ill applicants, through automatic annotation of health records accompanying new claims. We explore improved prioritization instead of fully autonomous claims approval.
Materials and Methods: We developed a sample of claims containing medical records at the moment of arrival in a single office. A series of tools annotated both patient records and public Web page descriptions of CAL medical conditions. We trained random forests to identify CAL patients and validated each model with 10-fold cross validation.
Results: Our main model, a general CAL classifier, had an area under the receiver operating characteristic curve of 0.915. Combining this classifier with existing software improved sensitivity from 0.960 to 0.994, detecting every deceased patient, but reducing positive predictive value to 0.216.
Discussion: True positive CAL identification is a priority, given CAL patient mortality. Mere prioritization of the false positives would not create a meaningful burden in terms of manual review. Death certificate data suggest the presence of truly ill patients among putative false positives.
Conclusion: To a limited extent, it is possible to identify gravely ill Social Security disability applicants by analyzing annotations of unstructured electronic health records, and the level of identification is sufficient to be useful in prioritizing case reviews.
OBJECTIVE
Every year, thousands of patients die waiting for disability benefits from the Social Security Administration (SSA).1 Some qualify for expedited service under SSA’s Compassionate Allowance (CAL) initiative, but CAL software relies heavily upon patients’ spelling abilities, focusing exclusively on information from a single field on the disability application.2 This paper describes the development of a supplemental process for identifying some overlooked but gravely ill Social Security disability applicants, through automatic annotation of the electronic health records often accompanying new disability claims. We use this method to explore improved prioritization instead of fully autonomous claims approval.
BACKGROUND AND SIGNIFICANCE
The SSA is a major source of support for disabled Americans, providing cash benefits for more than 14 million people.3 Each year, millions of new Social Security disability claims work their way through federal and state agencies, with a final award rate of 36%.4 Some awards are relatively straightforward, because prognoses are poor and relevant regulations require minimal medical evidence to establish eligibility; the CAL initiative exists to identify and expedite these claims.2 Recent research has employed survival analysis to expand the list of predetermined CAL conditions, providing more applicants with benefits before death.1 In 2014, 6.6% of claims qualified for expedited processing under CAL and other fast-track initiatives.5
Typically, cases receive CAL designation because SSA text-matching software finds reasonably accurate spellings of qualifying diseases, such as glioblastoma multiforme,6 in a specific field on the electronic disability application.2 The CAL software does not consider any of the unstructured medical records that may accompany a new claim, such as visit notes, consultations, operative notes, pathology reports, or discharge summaries. Research has already established the feasibility of text mining and natural language processing (NLP) for analysis of these unstructured records, providing insight into risk factors,7,8 disease-related language,9–12 diagnoses,13 treatment,14–16 disease progression,17 longitudinal patterns of pain severity,18 body site severity,19 phenotype-level severity,20 adverse medication events,21,22 and disease outbreaks.23 Some of this research involves electronic health records managed by another federal agency, the Department of Veterans Affairs.11,13,15,24,25 We explored NLP for annotating health records and identifying very ill disability applicants (Figure 1). MATERIALS AND METHODS
Sample development
We focused on new disability claims arriving at Minnesota Disability Determination Services, which handles ∼1.3% of the national SSA disability claims workload.26 Our sample included claims arriving within 1 year of January 10, 2014, when the SSA made many updates to CAL guidelines. While we received 33 420 new disability claims during this period, we studied only the 4655 claims arriving with at least 1 death certificate or medical record in TIFF file format. We gathered information about each claim, including whether SSA software noticed CAL language within applicant self-reports of disabling medical conditions. We also identified the organ system associated with each patient’s primary diagnosis. SSA algorithms designated 177 of these cases as CAL claims, but manual changes by staff suggested the presence of 8 false positives and 7 false negatives. Therefore, we had a single domain expert review each electronic health record and establish whether the case warranted CAL designation. This revealed a total of 181 CAL patients, and most of these were cancer patients (Table 1). We reviewed a total of 29 death certificates, identifying 6 for patients who died after applying for disability due to cancer, early-onset dementia, bowel obstruction, drug overdose, or suicide.
Model development
We developed predictors by reconciling patient records with public Web page descriptions of CAL medical conditions (Figure 2). First, we used optical character recognition software to extract text from 6824 faxed medical documents. Next, we used MetaMap,27 which includes an option for NegEx28 negation detection, to annotate all 4655 electronic health records and all 277 publicly available CAL condition descriptions.6 Annotations consisted of Medical Subject Heading29 concepts from 39 categories in the Unified Medical Language System Metathesaurus30 (Supplementary Material). The GeneralConText31 package allowed us to identify and eliminate annotations involving hypotheticals or references to persons other than the patient. We counted the number of times each annotation code appeared in each patient’s electronic health record. However, we only considered annotation codes present in annotations of CAL Web pages.
We used R32 to complete model development. After stratifying our data by CAL classification,33 we eliminated annotation codes that did not appear in at least 5% of 181 CAL cases. We then combined the annotation codes with 4 additional variables – birth year, sex, death certificate presence, and total page count, which we imputed for 189 patients; this amounted to 4% of our sample. We completed multidimensional scaling with randomForest,34 seeking insight into potential spatial distinctions between CAL and non-CAL cases, considering both unbalanced and balanced samples of CAL and non-CAL cases. For model training, we used the random stratified sampling feature in randomForest. This downsampling approach balanced our dataset by selecting all 181 CAL patients and random sets of 181 non-CAL patients for development of each decision tree. However, since the forest contained 100 such randomly generated trees, the whole forest covered our entire sample of 4655 patients. First, we trained a random forest model on the whole dataset. Then, since most CAL cases in our sample involved cancer, we split the dataset into cancer and noncancer cases, producing 2 additional random forests. To guide eventual interpretation of each model, we recorded both mean decrease in accuracy and mean decrease in Gini impurity, which show different responses to predictor correlation and scales of measurement.35 We tuned model parameters via grid searches36 and used the results from each model’s 10-fold cross-validation to generate and analyze receiver operating characteristic (ROC) curves.37 This approach allowed us to leverage all of our data for model training; however, it also produced meaningful validity statistics, since the cross-validation process generated separate training and testing subsets.
DISCUSSION
Our results suggest that, to a limited extent, it is possible to identify gravely ill patients applying for Social Security disability by analyzing annotations of unstructured electronic health records. We extracted meaningful text from thousands of faxed images, despite the occasional wrinkle or food stain. Using publicly available information, we created new, cross-validated CAL classifiers.
Cancer cases dominated our CAL sample. Most CAL cases involved cancer, and cancer-related language produced most of the true positives in the generalized CAL classifier. Cancer may have indirectly produced the musculoskeletal group of false positives, since the models were trained on cancer patients with other age-related comorbidities. We obtained a higher area under the ROC curve when predicting only cancer-related CAL cases, and many of the most important predictor variables were related to cancer diagnosis, staging, or treatment. The constellation of annotation codes referencing neoplasm, computerized tomography, magnetic resonance imaging, chemotherapy, and radiation therapy suggested additional cancer-related meaning for some otherwise nonspecific annotations, including emesis, anemia, pneumonia, and neurologic disturbance. References to dates, histories, symptoms, and examinations had obvious analogues in physician note headers, and particular combinations of these may reflect the nature of a patient’s contact with the health care system. However, another group of annotation codes had no clear connection to CAL or cancer. We hypothesize that references to teaching, education, writing, children, and parents resulted from processing of detailed individualized education programs for school-age youth who did not meet CAL requirements. For example, the teenagers in our sample were probably not born with catastrophic congenital abnormalities such as alobar holoprosencephaly, and had also not smoked enough cigarettes to develop small-cell lung cancer. Instead, they might suffer from common medical conditions that rarely meet stringent eligibility requirements. Some pediatric patients in our sample might have few limitations from mild, intermittent asthma that happens to respond to an inhaler. Similarly, some adults in our sample might function well with medication for hypertension and hypothyroidism. Our classifiers had difficulty making predictions involving other, more grave, nonneoplastic CAL medical conditions; cleaner input, larger sample sizes, and longer time periods could help identify more patients with rare noncancerous conditions.
Our methods improved overall CAL detection when combined with current CAL software. With the identification of 11 additional CAL patients, sensitivity improved from 0.960 to 0.994. This combination of methods produced more false positives, and most of these had no obvious cause. This decreased the positive predictive value, but identification of true positives is a priority, given mortality rates for CAL medical conditions. Moreover, false positives create no meaningful burden in terms of manual review. This is because each case receives careful manual analysis, regardless of CAL classification. The CAL designation simply changes the order in which patients receive service. Similar methods could improve prioritization for other lengthy SSA processes, based on identification of factors such as illness or homelessness; examples include quality reviews, periodic reviews, judicial appeals, repeat applications, and fraud investigations. This suggests a class of use cases, in which a new system could be helpful despite limited positive predictive value, because it imposes only minimal burdens. Of course, future research should explore automation of the entire approval process, which would reduce the burden of manual review.
The deceased patients in our sample merit special attention. Death does not necessarily eliminate eligibility for SSA disability benefits, which can be an important source of financial relief for surviving spouses and children. The presence of a death certificate proved to be an important CAL predictor variable, but closer inspection of false positive classifications revealed disproportionately large numbers of deceased patients. This contradiction, along with an abundance of neoplastic false positive predictions, suggests that our classifiers identified patients who were truly ill, though they did not quite meet CAL criteria. Such patients may deserve expedited service, even if they fall short of restrictive CAL guidelines. The false positive classifications may therefore reflect a mixture of predictive error and dichotomization of essentially continuous variables related to mortality. Our methods help address this concern by providing a continuous score that incorporates data from an entire health record, instead of the binary result from string-matching against a single application form field.
Existing CAL software may overlook patients for a number of reasons. Some of the sickest patients do not speak English well, if they are able to speak at all. Even well-educated native English speakers with clinical expertise can misspell the names of rare, eponymous conditions such as Creutzfeldt-Jakob disease. Language skills aside, current software overlooks other patients who communicate well but underestimate the level of detail necessary to trigger CAL rules. For example, mesothelioma patients might not specify their disease as peritoneal or pleural. Application questions and help manuals should be revised to minimize these problems, but software should also be made to consider more detailed and reliable clinical information that might already be available in the form of copied health records. Small improvements in these areas could benefit large numbers of patients on an annual basis and a nationwide scale.
We estimate the yearly impact of nationwide implementation using data from Table 2. Current software detected 177 true and false positives, including 169 living patients and 8 deceased patients. By contrast, the combination of current and proposed CAL detection methods yielded 839 true and false positives, including 810 living patients and 29 deceased patients. This amounts to 641 additional living patients and 21 additional deceased patients. Since Minnesota Disability Determination Services handles 1.3% of the national workload,26 we estimate that nationwide implementation of our proposed methods would expedite service for approximately 49 000 living patients and 1600 patients who die by the beginning of case processing. We hypothesize that these CAL classifiers would help many patients who die later, after state agencies begin collecting health records but before they receive a final decision from a federal office. The value of this extra CAL detection layer would increase as more patients and grieving family members include medical reports with disability applications.
CONCLUSION
To a limited extent, it is possible to identify gravely ill Social Security disability applicants by analyzing annotations of unstructured electronic health records, and the level of identification is sufficient to be useful in prioritizing case reviews. On an annual basis and a nationwide scale, these methods could benefit many living patients, as well as thousands who die before the beginning of case processing.
"""

txt = instruction1 + matn + '[/INST]'

In [35]:
import json
from pathlib import Path

dimension = "objectives"

# Define model and adapter paths
model_path = "mistralai/Mistral-7B-Instruct-v0.2"
adapter_path = Path(f"Models/adapters_{dimension}")  # Convert to Path object

# Define LoRA configuration
lora_config = {
    "lora_layers": 8,
    "num_layers": 12,  # Replace with the actual number of layers for your model
    "lora_parameters": {
        "rank": 8,
        "scale": 10.0,
        "dropout": 0.1,
    },
}

# Save the LoRA config to the adapter path
with open(adapter_path / "adapter_config.json", "w") as fid:
    json.dump(lora_config, fid, indent=4)

# Load model and tokenizer using the load function
model2, tokenizer2 = load(
    path_or_hf_repo=model_path,
    adapter_path=adapter_path
)


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [37]:

instructions = {
    "objectives": "[INST]You are an expert in objective extraction from summary of papers. What is the objective of the following full text? Just tell me in one sentence and do not repeat your answer and summarize your answer, do not give me unfinished sentences or sentences related to method, I only want objective of the paper:",
    "AI_app": "[INST]You are an expert in data extraction of systematic reviews, What is the AI application used for the disable population in the following full-text based on this article?(for example predictions, decision-making, physical-aide, classification, clustering), just name the application from options that i gave to you in one word:",
    "Disability_type": "[INST] You are an expert in data extraction of systematic reviews, given the following which disability impairement does this text works on (between vision, cognition, communication, auditive, and motor)? Just select from the options I gave to you: ",
    "Barriers": "[INST] You are an expert in data extraction of systematic reviews, given the following which disability impairement does this text works on (between vision, cognition, communication, auditive, and motor)? Just select from the options I gave to you: ",
    "Facilitators" : "[INST] You are an expert in data extraction of systematic reviews, what is the facilitators of of the following full-text to inlcude people with handicap in text based on this article?, for example I have considered these items as the facilitators in the studies if they: Extract information on the facilitators for including people with disabilities in AI research mentioned in each paper, such as: + Collaboration with disability organizations and advocates + Training and expertise in accessibility and inclusivity + Adaptive data collection methods (e.g., online surveys) + Inclusive participant recruitment strategies + Facilitate interacting with computers by providing an assistive devices + Using technology for detection of problems among people with disability just give it in one short sentence no additional words, if they have not addressed the facilitatiors just say : not addressed" ,
    "Equity" : "[INST] You are an expert in data extraction of systematic reviews, Is equity considered in the following full-text to inlcude people with handicap in  text based on this article?,for example I have considered these items as considering equity: 1. Diversity in sample consideration 2. Promoting equity in access to technology and computer interaction 3. Including nation-wide registry as the sample data 4. Involving usability testing with people with disabilities to ensure that AI solutions meet their needs. 5. Help in developing policies to improve the health of this vulnerable population. 6. Ensuring that all patients have equitable access to the technology. just give it in one short sentence no additional words",
    "Tools" : "[INST] You are an expert in data extraction of systematic reviews, Extract information on the specific applications or tools developed using AI that aim to improve the lives of persons with disabilities in the following full-text  This could include, but is not limited to: + Assistive technologies for daily living tasks (e.g., automated door openers, smart home devices) + Communication aids (e.g., speech-to-text systems, sign language recognition) + Diagnostic tools (e.g., image analysis software for visual impairments, audio processing software for hearing impairments) + Rehabilitation and therapy tools (e.g., virtual reality-based physical therapy, AI-powered cognitive training) just give it in one short phrase no additional explanations",
    "Disability_outcome" : "[INST] You are an expert in data extraction of systematic reviews,Extract information on disability-specific outcomes mentioned in the following full-textsuch as improved communication skills for individuals with speech impairments but not limited to this, search it based the text I gave you. just give it in one short phrase no additional explanations",
    "Implications" : "[INST] You are an expert in data extraction of systematic reviews, what is the Implications of this study to inlcude people with handicap in the following full-text based on this article?, just give it in one short sentence no additional words"

}

# Select instruction
instruction = instructions[dimension]
txt = instruction + matn + ' [/INST]'



In [38]:
tx1 = generate(model2, tokenizer2, txt, max_tokens=200)
tx1


'The objective of the paper is to develop a supplemental process for identifying some overlooked but gravely ill Social Security disability applicants, through automatic annotation of the electronic health records often accompanying new disability claims. The study explores improved prioritization instead of fully autonomous claims approval.'